In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import pathlib

import IPython.display as display
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from PIL import Image

data_dir = pathlib.Path(r'C:\Users\jadav\Desktop\Tensorflow notebooks\cats-v-dogs\testing')

AUTOTUNE = tf.data.experimental.AUTOTUNE

CLASS_NAMES = np.array([item.name for item in data_dir.glob("*")])
print(CLASS_NAMES)
BATCH_SIZE = 32
IMG_HEIGHT = 224
IMG_WIDTH = 224
# for f in list_ds.take(1):
#     print(f)
# dataset = tf.data.Dataset.from_tensor_slices(list_ds)
# print(dataset)

['cats' 'dogs']


In [2]:
list_ds = tf.data.Dataset.list_files(str(data_dir / '*/*'))

img = list_ds.take(1)
img = list(img.as_numpy_iterator())

file = tf.io.read_file(img)

InvalidArgumentError: Input filename tensor must be scalar, but had shape: [1] [Op:ReadFile]

In [ ]:
def get_label(file_path):
    parts = tf.strings.split(file_path, "/")
    return parts[-2] == CLASS_NAMES

In [ ]:
def decode_img(img):
    img = tf.image.decode_jpeg(img, channels=3)  # color images
    img = tf.image.convert_image_dtype(img, tf.float32)
    # convert unit8 tensor to floats in the [0,1]range
    return tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT])

img = decode_img(img)
# resize the image into 224*224

In [ ]:
def process_path(file_path):
    label = get_label(file_path)
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img, label

In [ ]:
list_ds = tf.data.Dataset.list_files(str(data_dir / "*/*"))

print(list_ds)

for f in list_ds.take(1):
  print(f)

labeled_ds = list_ds.map(process_path, num_parallel_calls=AUTOTUNE)

print(labeled_ds)

In [ ]:
# for image, label in labeled_ds:
#     print("Image shape: ", image.numpy().shape)
#     print("Label: ", label.numpy())

In [ ]:
def prepare_for_training(ds, cache=True, shuffle_buffer_size=1000):
  
  if cache:
    if isinstance(cache, str):
      ds = ds.cache(cache)
    else:
      ds = ds.cache()
  ds = ds.shuffle(buffer_size=shuffle_buffer_size)
  ds = ds.repeat() #repeat forever
  ds = ds.batch(BATCH_SIZE)
  ds = ds.prefetch(buffer_size=AUTOTUNE)
  return ds

In [ ]:
train_ds = prepare_for_training(labeled_ds)
print(train_ds)
image_batch, label_batch = next(iter(train_ds))
show_batch(image_batch.numpy(), label_batch.numpy())